<a href="https://colab.research.google.com/github/emredeveloper/Transformers--General-AI/blob/main/Mixture_of_Experts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Expert(nn.Module):
    """Tek bir uzmanın basit ileri beslemeli ağı"""
    def __init__(self, input_dim, hidden_dim):
        super(Expert, self).__init__()
        self.ffn = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim)
        )

    def forward(self, x):
        return self.ffn(x)

class Router(nn.Module):
    """Yönlendirici: Hangi uzmanın etkinleştirileceğine karar verir."""
    def __init__(self, input_dim, num_experts):
        super(Router, self).__init__()
        self.gate = nn.Linear(input_dim, num_experts)

    def forward(self, x):
        # Uzmanlar için olasılık hesaplama
        return F.softmax(self.gate(x), dim=-1)

class MoELayer(nn.Module):
    """Mixture of Experts katmanı"""
    def __init__(self, input_dim, hidden_dim, num_experts, top_k=2):
        super(MoELayer, self).__init__()
        self.num_experts = num_experts
        self.top_k = top_k
        self.experts = nn.ModuleList([Expert(input_dim, hidden_dim) for _ in range(num_experts)])
        self.router = Router(input_dim, num_experts)

    def forward(self, x):
        batch_size, seq_len, _ = x.size()  # Boyutları ayıkla
        x_flat = x.view(-1, x.size(-1))  # Batch ve seq_len birleştir

        # Yönlendirici tarafından uzman seçimi
        route_weights = self.router(x_flat)
        topk_weights, topk_indices = torch.topk(route_weights, self.top_k, dim=-1)

        # Uzmanların çıktılarının birleştirilmesi
        outputs = torch.zeros_like(x_flat)
        for i in range(self.top_k):
            weight = topk_weights[:, i].unsqueeze(-1)
            expert_idx = topk_indices[:, i]
            outputs += weight * torch.cat(
                [self.experts[expert](x_flat[j].unsqueeze(0)) for j, expert in enumerate(expert_idx)], dim=0
            )

        # Orijinal boyuta geri dön
        outputs = outputs.view(batch_size, seq_len, -1)
        return outputs

class MoETransformer(nn.Module):
    """MoE içeren basit bir Transformer"""
    def __init__(self, input_dim, hidden_dim, num_heads, num_experts, top_k):
        super(MoETransformer, self).__init__()
        self.attention = nn.MultiheadAttention(embed_dim=input_dim, num_heads=num_heads, batch_first=True)
        self.moe_layer = MoELayer(input_dim, hidden_dim, num_experts, top_k)
        self.norm1 = nn.LayerNorm(input_dim)
        self.norm2 = nn.LayerNorm(input_dim)

    def forward(self, x):
        # Multi-head attention
        attn_output, _ = self.attention(x, x, x)
        x = self.norm1(x + attn_output)

        # Mixture of Experts katmanı
        moe_output = self.moe_layer(x)
        x = self.norm2(x + moe_output)

        return x

# Örnek kullanım
input_dim = 128
hidden_dim = 256
num_heads = 4
num_experts = 3
top_k = 2
seq_len = 10
batch_size = 5

# Model oluşturma
model = MoETransformer(input_dim, hidden_dim, num_heads, num_experts, top_k)

# Rastgele giriş verisi
x = torch.rand(batch_size, seq_len, input_dim)

# Çıktı
output = model(x)
print("Çıkış boyutu:", output.shape)

Çıkış boyutu: torch.Size([5, 10, 128])
